In [1]:
import networkx as nx
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import random as rnd
%matplotlib inline

In [2]:
def predictor_Baseline(): # Our baseline predictor, taken from the PS5 solutions
    return rnd.random()

def predictor_Jaccard(Gx,i,j): # Our Jaccard predictor, taken from the PS5 solutions
    j_temp = nx.jaccard_coefficient(Gx, [(i,j)])
    for u,v,p in j_temp:
        jaccard = p
    n = Gx.order()
    return jaccard + rnd.uniform(0,1/(10*n))

def predictor_DegreeProd(Gx,i,j): # Our Jaccard predictor, taken from the PS5 solutions
    d_temp = nx.preferential_attachment(Gx, [(i,j)])
    for u,v,p in d_temp:
        degreeprod = p
    n = Gx.order()
    return degreeprod + rnd.uniform(0,1/(10*n))

def predictor_iforget(Gx,i,j): # Our <insert> predictor, created by us
    pass

def predictor_randomwalk(Gx,i,j): # Our <insert> predictor, created by us
    pass

def get_candidateEdges(G): # This function returns all the non edges in graph G
    nonEdges = list(nx.non_edges(G))
    return nonEdges

def apply_Predictors(G, Y): # This functions stores the results from all of our predictors in an np array 
    Y2 = get_candidateEdges(G) # First, we get the edges that we want our predictors to make a prediction on
    table = np.ndarray((len(Y2), 8), dtype = object) # Here, we are creating the np array to store all of our results beforehand, this is done to speed up program runtime
    table[:,0] = int(len(Y2)) # We make sure to cast the int type to the first two columns
    table[:,1] = int(len(Y2)) # These first two columns will store the nodes within the edge, we need them in int form in order to check if they are in the missing edge list
    table[:,2] = int(len(Y2)) # We cast the third column to int because this will store the tau values, 0 = False and 1 = True
    table[:,3:] = float(len(Y2)) # The rest of the columns are casted to float format because they will contain the results of our predictors
    counter = 0 # This counter will be used to add an entry to the closest vacant spot in our np array

    for link in Y2: # We iterate through the candidate edges, having our predictors make an estimation on the liklihood of an edge being a real missing edge 
        if(link[0], link[1]) in Y: # We check if the candidate edge is a true missing edge 
            tau = 1
        else:
            tau = 0 # If the candidate edge is not within the missing edge list, then tau = 0
        baseline = float(predictor_Baseline()) # We make sure to cast the results of our predictors to floats
        jaccard = float(predictor_Jaccard(G, link[0], link[1])) # We do this because the output of the predictors are initially strings, which do not work within a homogenous np array
        degreeProd = float(predictor_DegreeProd(G, link[0], link[1]))
        NewPredictor1 = 0 # Kurt, here is where you will make the function call to the new predictors, make sure the return of the function is a float 
        NewPredictor2 = 0
        
        table[counter,:] = [link[0], link[1], tau, baseline, jaccard, degreeProd, NewPredictor1, NewPredictor2] # Here we are adding an entry to the np array in the closest vacant spot
        counter += 1 # Increment counter to the index of the closest vacant spot

    return table

In [3]:
G = nx.read_edgelist("ChCh-Miner_durgbank-chem-chem.tsv") # Our dataset is stored as an edge list, we use networkx to read the file into a Graph object

def generateGo(G, alpha): # This function creates the observed graph from the original graph
    Go = G.copy() # We first make a copy of the original graph
    mapping = {old_label:new_label for new_label, old_label in enumerate(Go.nodes())} # The nodes in this graph are in this form: XX00000, this code is modified from stackoverflow
    Go = nx.relabel_nodes(Go, mapping) # We relabel the node names into integers because np arrays are homogenous data structures and cannot contain strings and numbers 
    missingEdges = [] # This list will contain all the missing edges which remove from Go
    randomNum = 0
    
    for edge in Go.edges(): # We iterate through the edges of Go
        randomNum = np.random.uniform(0, 1) # We generate a random number between 0 and 1
        if randomNum > alpha: # We use alpha where 0 < alpha < 1 to determine how many edges will be removed from the Go
            Go.remove_edge(edge[0], edge[1]) 
            missingEdges.append(edge) # We make sure to store the edge which was removed 
    
    return Go, missingEdges

Go, missing = generateGo(G, .87) 
table = apply_Predictors(Go, missing) 

In [4]:
def tabulate_TPR_FPR(S): # This code was used from our PS5
    tkTrue = 0
    tkFalse = 0
    TPR = []
    FPR = []
    T = 0
    F = 0
    
    for i in range(0, len(S)): # Iterate once through S in order to determine the normalizing constants
        if S[i][2] == 1:
            T += 1
        else:
            F += 1
            
    for j in range(0, len(S)): # Iterate again to calculate the TPR and FPR
        if(S[j][2]) == 1:
            tkTrue += 1
        else:
            tkFalse += 1
        TPR.append(tkTrue / T)
        FPR.append(tkFalse / F)
        
    columnOne = S[:, 0] # Here we are getting the columns of the table in the desired return format
    columnTwo = S[:, 1]
    columnThree = S[:, 2]
    columnFour = np.array(TPR)
    columnFive = np.array(FPR)
    columnSix = S[:, 3]

    columnOne = columnOne.reshape(len(columnOne), 1) # Reshaping columns, turning them into columns instead of horizontal lists
    columnTwo = columnTwo.reshape(len(columnTwo), 1)
    columnThree = columnThree.reshape(len(columnThree), 1)
    columnFour = columnFour.reshape(len(columnFour), 1)
    columnFive = columnFive.reshape(len(columnFive), 1)
    columnSix = columnSix.reshape(len(columnSix), 1)

    table = np.hstack((columnOne, columnTwo)) # Here we are combining those columns into the table in the desired return format
    table = np.hstack((table, columnThree))
    table = np.hstack((table, columnFour))
    table = np.hstack((table, columnFive))
    table = np.hstack((table, columnSix))
    return table    

def calculate_AUC(TPR,FPR):
    AUC = 0
    TPR = np.insert(TPR, 0, 0) # Prepend 0
    FPR = np.insert(FPR, 0, 0)
    for i in range(1, len(TPR)):
        AUC += TPR[i] * (FPR[i] - FPR[i - 1]) # calculate AUC according to lecture notes 4
    return AUC

In [5]:
columnOne = table[:, 0] # Here I am getting the columns of the table in the desired input format, this code was taken from our PS5 
columnTwo = table[:, 1] # We use the columns to create tables in the format needed for the tabulate_TPR_FPR and calculate_AUC function
columnThree = table[:, 2] # The first three columns are node i, node j, tau, in that order
columnFour = table[:, 3] # This column contains the baseline predictions
columnFive = table[:, 4] # This column contains the jaccard predictions
columnSix = table[:, 5] # This column contains the degree product predictions
# columnSeven = table[:, 6] # This column contains the <insert> predictions
# columnEight = table[:, 7] # This column contains the <insert> predictions

columnOne = columnOne.reshape(len(columnOne), 1) # Reshaping columns, turning them into columns instead of horizontal lists
columnTwo = columnTwo.reshape(len(columnTwo), 1)
columnThree = columnThree.reshape(len(columnThree), 1)
columnFour = columnFour.reshape(len(columnFour), 1)
columnFive = columnFive.reshape(len(columnFive), 1)
columnSix = columnSix.reshape(len(columnSix), 1)
# columnSeven = columnThree.reshape(len(columnSeven), 1)
# columnEight = columnFour.reshape(len(columnEight), 1)

baselineTable = np.hstack((columnOne, columnTwo)) # Here we are combining the columns of the table in the desired return format
baselineTable = np.hstack((baselineTable, columnThree))
baselineTable = np.hstack((baselineTable, columnFour))
baselineTable = baselineTable[np.argsort(baselineTable[:, 3])[::-1]] # Here we are sorting the table by the score column in descending order

jaccardTable = np.hstack((columnOne, columnTwo)) 
jaccardTable = np.hstack((jaccardTable, columnThree)) 
jaccardTable = np.hstack((jaccardTable, columnFive))
jaccardTable = jaccardTable[np.argsort(jaccardTable[:, 3])[::-1]]

degreeProductTable = np.hstack((columnOne, columnTwo)) 
degreeProductTable = np.hstack((degreeProductTable, columnThree))
degreeProductTable = np.hstack((degreeProductTable, columnSix))
degreeProductTable = degreeProductTable[np.argsort(degreeProductTable[:, 3])[::-1]]

# newPredictorOneTable = np.hstack((columnOne, columnTwo)) 
# newPredictorOneTable = np.hstack((newPredictorOneTable, columnThree))
# newPredictorOneTable = np.hstack((newPredictorOneTable, columnSix))
# newPredictorOneTable = newPredictorOneTable[np.argsort(newPredictorOneTable[:, 3])[::-1]]

# newPredictorTwoTable = np.hstack((columnOne, columnTwo)) 
# newPredictorTwoTable = np.hstack((newPredictorTwoTable, columnThree))
# newPredictorTwoTable = np.hstack((newPredictorTwoTable, columnSix))
# newPredictorTwoTable = newPredictorTwoTable[np.argsort(newPredictorTwoTable[:, 3])[::-1]]

baselineTable = tabulate_TPR_FPR(baselineTable) # Here we are tabulating the tables and getting the TPR and FPR 
jaccardTable = tabulate_TPR_FPR(jaccardTable)
degreeProductTable = tabulate_TPR_FPR(degreeProductTable)
# newPredictorOneTable = tabulate_TPR_FPR(newPredictorOneTable)
# newPredictorTwoTable = tabulate_TPR_FPR(newPredictorTwoTable)

baselineAUC = calculate_AUC(baselineTable[:, 3], baselineTable[:, 4]) # Once we have tabulated the tables, we get the AUC for each predictor
jaccardAUC = calculate_AUC(jaccardTable[:, 3], jaccardTable[:, 4])
degreeProductAUC = calculate_AUC(degreeProductTable[:, 3], degreeProductTable[:, 4])
# predictorOneAUC = calculate_AUC(newPredictorOneTable[:, 3], newPredictorOneTable[:, 4])
# predictorTwoAUC = calculate_AUC(newPredictorTwoTable[:, 3], newPredictorTwoTable[:, 4])

ValueError: Unrecognized location 'Lower Right'. Valid locations are
	best
	upper right
	upper left
	lower left
	lower right
	right
	center left
	center right
	lower center
	upper center
	center


In [8]:
text = "Baseline AUC: {:.3f} \n\nJaccard AUC: {:.3f} \n\nDegree Product AUC: {:.3f} \n\nNP1 AUC: {:.3f} \n\nNP2 AUC: {:.3f}".format(baselineAUC, jaccardAUC, degreeProductAUC, 0, 0)
fig = plt.figure(figsize = (16, 8)) # This code is taken from the PS5 solutions and our own PS5
ax1 = fig.add_subplot(111)
plt.title("FPR vs. TPR For All Predictors")
plt.plot(baselineTable[:,4], baselineTable[:,3], 'bo-', alpha = 0.5, label = 'Baseline')
plt.plot(jaccardTable[:,4], jaccardTable[:,3], 'rs-', alpha = 0.5, label = 'Jaccard')
plt.plot(degreeProductTable[:,4], degreeProductTable[:,3], 'gv-', alpha = 0.5, label = 'Degree Product')
# plt.plot(newPredictorOneTable[:,4], newPredictorOneTable[:,3], 'o1-', alpha = 0.5, label = 'New Predictor One')
# plt.plot(newPredictorTwoTable[:,4], newPredictorTwoTable[:,3], 'pD-', alpha = 0.5, label = 'New Predictor Two')
plt.annotate(text, xy = (0.75, 0.35), xycoords = "axes fraction", bbox = dict(facecolor = 'none', edgecolor = 'black', boxstyle = 'round, pad = 1'))
plt.legend(loc = "lower right")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()